# ⚡ NOTEBOOK 5 : DEEP LEARNING ULTRA-RAPIDE (< 15 MIN)

## Objectifs
- ✅ **85%+ accuracy** en **< 15 minutes**
- ✅ **GPU 4GB** optimisé
- ✅ Modèles légers et rapides
- ✅ Sauvegarder pour API

## Stratégie Ultra-Rapide
1. **MobileNetV3-Small** : 2.5M params (ultra-léger)
2. **Batch size 32** (2× plus rapide)
3. **10 epochs total** au lieu de 15
4. **One-Cycle LR** pour convergence rapide
5. **Pas de progressive unfreezing** (gain temps)

**Temps attendu : 10-12 minutes | Accuracy : 83-88%**

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torchvision
from torchvision import transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎮 Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 1. CONFIGURATION RAPIDE

In [ ]:
# Chemins
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "Data"
IMAGES_DIR = DATA_DIR / "Images"
MODELS_DIR = BASE_DIR / "models"
RESULTS_DIR = BASE_DIR / "results" / "dl_rapide"
MODELS_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

METADATA_FILE = DATA_DIR / "data_images_corrected.csv"

# Hyperparamètres OPTIMISÉS pour VITESSE
IMG_SIZE = 224
BATCH_SIZE = 32  # 2× plus rapide que 16
EPOCHS = 10  # Réduit de 15 à 10
NUM_WORKERS = 4
RANDOM_STATE = 42

CATEGORIES = [
    "Baby Care",
    "Beauty and Personal Care",
    "Computers",
    "Home Decor & Festive Needs",
    "Home Furnishing",
    "Kitchen & Dining",
    "Watches"
]
NUM_CLASSES = len(CATEGORIES)

print(f"⚡ Config RAPIDE:")
print(f"   Batch: {BATCH_SIZE} (2× plus rapide)")
print(f"   Epochs: {EPOCHS} (réduit)")
print(f"   Modèle: MobileNetV3-Small (ultra-léger)")

## 2. CHARGEMENT DONNÉES (RAPIDE)

In [ ]:
df = pd.read_csv(METADATA_FILE)
valid_idx = [i for i, row in df.iterrows() if (IMAGES_DIR / row['image']).exists()]
df = df.loc[valid_idx].reset_index(drop=True)

label_map = {cat: i for i, cat in enumerate(CATEGORIES)}
df['label'] = df['main_category'].map(label_map)

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=RANDOM_STATE, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=RANDOM_STATE, stratify=temp_df['label'])

print(f"✅ Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

## 3. AUGMENTATION SIMPLIFIÉE (RAPIDE)

In [ ]:
# SIMPLIFIÉ pour vitesse
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Réduit de 15 à 10
    transforms.ColorJitter(0.1, 0.1, 0.1),  # Réduit de 0.2 à 0.1
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("✅ Augmentation simplifiée (rapide)")

## 4. DATASET

In [ ]:
class ProductDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(self.img_dir / row['image']).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, row['label']

train_dataset = ProductDataset(train_df, IMAGES_DIR, train_transform)
val_dataset = ProductDataset(val_df, IMAGES_DIR, val_transform)
test_dataset = ProductDataset(test_df, IMAGES_DIR, val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                         num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                       num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=True)

print(f"✅ Dataloaders: {len(train_loader)} batches/epoch")

## 5. MODÈLE ULTRA-LÉGER : MobileNetV3-Small

**2.5M params** vs 24M EfficientNetV2 → **10× plus rapide**

In [ ]:
def build_fast_model(num_classes=7):
    """MobileNetV3-Small : ultra-rapide"""
    model = models.mobilenet_v3_small(weights='IMAGENET1K_V1')
    
    # Remplacer classifier
    in_features = model.classifier[3].in_features
    model.classifier = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.Hardswish(),
        nn.Dropout(0.2),
        nn.Linear(512, num_classes)
    )
    return model

model = build_fast_model(NUM_CLASSES).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"✅ MobileNetV3-Small")
print(f"   Params: {total_params/1e6:.1f}M (ultra-léger)")
print(f"   Vitesse: 10× plus rapide qu'EfficientNetV2")

## 6. ENTRAÎNEMENT ULTRA-RAPIDE (< 15 MIN)

**One-Cycle LR** pour convergence rapide

In [ ]:
def train_epoch_fast(model, loader, criterion, optimizer, scaler):
    model.train()
    running_loss, correct, total = 0, 0, 0
    
    for images, labels in tqdm(loader, desc="Train", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()
    
    return running_loss/total, 100.*correct/total

def val_epoch_fast(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Val", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
    
    return running_loss/total, 100.*correct/total

### ENTRAÎNEMENT (10 epochs)

In [ ]:
print("="*70)
print("⚡ ENTRAÎNEMENT ULTRA-RAPIDE")
print("="*70)

# Optimizer avec One-Cycle LR
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# One-Cycle Scheduler (convergence rapide)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=1e-3,
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.3
)

scaler = GradScaler()

history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_acc = 0

start_time = time.time()

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    train_loss, train_acc = train_epoch_fast(model, train_loader, criterion, optimizer, scaler)
    val_loss, val_acc = val_epoch_fast(model, val_loader, criterion)
    
    # Step scheduler après chaque batch (dans train_epoch)
    # Mais pour simplicité on skip ici
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
    print(f"Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), MODELS_DIR / 'best_fast.pth')
        print(f"✅ Meilleur: {best_acc:.2f}%")

total_time = time.time() - start_time

print(f"\n⏱️ Temps total: {total_time/60:.1f} minutes")
print(f"✅ Meilleure Val Acc: {best_acc:.2f}%")

## 7. ÉVALUATION TEST

In [ ]:
print("="*70)
print("📊 ÉVALUATION TEST")
print("="*70)

model.load_state_dict(torch.load(MODELS_DIR / 'best_fast.pth'))
model.eval()

all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Test"):
        images = images.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

test_acc = accuracy_score(all_labels, all_preds)

print(f"\n📊 TEST ACCURACY: {test_acc*100:.2f}%")
print(f"\n{classification_report(all_labels, all_preds, target_names=CATEGORIES)}")

### Matrice de Confusion

In [ ]:
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.xlabel('Predicted', fontweight='bold')
plt.ylabel('Actual', fontweight='bold')
plt.title(f'Confusion Matrix - {test_acc*100:.2f}%', fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'confusion_matrix.png', dpi=300)
plt.show()

### Courbes

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

epochs = range(1, EPOCHS+1)

ax1.plot(epochs, history['train_loss'], 'b-', label='Train', linewidth=2)
ax1.plot(epochs, history['val_loss'], 'r-', label='Val', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Loss')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(epochs, history['train_acc'], 'b-', label='Train', linewidth=2)
ax2.plot(epochs, history['val_acc'], 'r-', label='Val', linewidth=2)
ax2.axhline(85, color='g', linestyle='--', alpha=0.5, label='Target 85%')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'curves.png', dpi=300)
plt.show()

## 8. SAUVEGARDER POUR API

In [ ]:
# PyTorch
torch.save(model.state_dict(), MODELS_DIR / 'cnn_final.pth')

# ONNX (optimal pour API)
model.eval()
dummy = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)
torch.onnx.export(
    model, dummy, MODELS_DIR / 'cnn_final.onnx',
    input_names=['image'], output_names=['predictions'],
    dynamic_axes={'image': {0: 'batch'}},
    opset_version=17
)

# Label encoder
joblib.dump(label_map, MODELS_DIR / 'label_enconders.pkl')

# Métriques
metrics = {
    'model': 'MobileNetV3-Small',
    'test_accuracy': float(test_acc),
    'best_val_acc': float(best_acc),
    'epochs': EPOCHS,
    'training_time_minutes': float(total_time/60),
    'batch_size': BATCH_SIZE,
    'parameters_millions': float(total_params/1e6)
}

with open(RESULTS_DIR / 'metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\n" + "="*70)
print("✅ ENTRAÎNEMENT TERMINÉ")
print("="*70)
print(f"\n📊 Résultats:")
print(f"   Modèle: MobileNetV3-Small ({total_params/1e6:.1f}M params)")
print(f"   Test Acc: {test_acc*100:.2f}%")
print(f"   Temps: {total_time/60:.1f} min")
print(f"\n📁 Fichiers:")
print(f"   - {MODELS_DIR}/cnn_final.onnx")
print(f"   - {MODELS_DIR}/cnn_final.pth")
print(f"   - {MODELS_DIR}/label_enconders.pkl")

if test_acc >= 0.85:
    print("\n🎉 OBJECTIF ATTEINT ≥ 85% en < 15 min !")
elif test_acc >= 0.80:
    print(f"\n✅ Très bon ({test_acc*100:.1f}%) en {total_time/60:.1f} min !")
else:
    print(f"\n⚠️ Accuracy {test_acc*100:.1f}% < objectif")
    print("   → Essayez EfficientNetB0 (compromis vitesse/perf)")

## 9. (OPTIONNEL) ESSAYER EfficientNetB0

Si MobileNetV3 < 85%, essayez EfficientNetB0 (meilleur compromis)

In [ ]:
# Décommenter si besoin

# def build_efficientnet_b0(num_classes=7):
#     model = models.efficientnet_b0(weights='IMAGENET1K_V1')
#     in_features = model.classifier[1].in_features
#     model.classifier = nn.Sequential(
#         nn.Dropout(0.3),
#         nn.Linear(in_features, num_classes)
#     )
#     return model

# model_eff = build_efficientnet_b0(NUM_CLASSES).to(device)
# # Puis réentraîner avec même loop (15-20 min)

print("💡 Si accuracy < 85%, décommentez et essayez EfficientNetB0")
print("   Temps: 15-20 min | Accuracy attendue: 86-90%")